# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **data analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `dataproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
#from matplotlib_venn import venn2

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject


interest = pd.read_csv('MEI_FIN_05042023142240461.csv')
interest.head()



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,SUBJECT,Subject,LOCATION,Country,FREQUENCY,Frequency,TIME,Time,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,SP,"Share Prices, Index",CAN,Canada,A,Annual,1974,1974,IDX,Index,0,Units,2015_100,2015=100,7.089914,NaN,NaN
1,SP,"Share Prices, Index",CAN,Canada,A,Annual,1975,1975,IDX,Index,0,Units,2015_100,2015=100,6.975263,NaN,NaN
2,SP,"Share Prices, Index",CAN,Canada,A,Annual,1976,1976,IDX,Index,0,Units,2015_100,2015=100,7.219618,NaN,NaN
3,SP,"Share Prices, Index",CAN,Canada,A,Annual,1977,1977,IDX,Index,0,Units,2015_100,2015=100,7.043305,NaN,NaN
4,SP,"Share Prices, Index",CAN,Canada,A,Annual,1978,1978,IDX,Index,0,Units,2015_100,2015=100,8.083440,NaN,NaN


# Read and clean data

Import your data, either through an API or manually, and load it. 

## Explore each data set

In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

**Interactive plot** :

In [ ]:
def plot_func():
    # Function that operates on data set
    pass

widgets.interact(plot_func, 
    # Let the widget interact with data through plot_func()    
); 


Explain what you see when moving elements of the interactive plot around. 

# Merge data sets

Now you create combinations of your loaded data sets. Remember the illustration of a (inner) **merge**:

In [ ]:
plt.figure(figsize=(15,7))
v = venn2(subsets = (4, 4, 10), set_labels = ('Data X', 'Data Y'))
v.get_label_by_id('100').set_text('dropped')
v.get_label_by_id('010').set_text('dropped' )
v.get_label_by_id('110').set_text('included')
plt.show()

Here we are dropping elements from both data set X and data set Y. A left join would keep all observations in data X intact and subset only from Y. 

Make sure that your resulting data sets have the correct number of rows and columns. That is, be clear about which observations are thrown away. 

**Note:** Don't make Venn diagrams in your own data project. It is just for exposition. 

# Analysis

To get a quick overview of the data, we show some **summary statistics** on a meaningful aggregation. 

MAKE FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

# Conclusion

ADD CONCISE CONLUSION.